In [21]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [46]:
import pyproj
from shapely import geometry
from shapely.ops import transform
from functools import partial

In [16]:
# Getting data

accidents = pd.read_csv("strassenverkehrsunfallorte.csv", sep=';')
accidents[["AccidentUID", "AccidentType", "Location"]]

,AccidentUID,AccidentType,Location
0,5CC0CF832F640198E0530A83942715F8,at2,"47.438312915107666,9.401879723536387"
1,5EA7539566F001CCE0530A839427A876,at0,"47.43746379938151,9.393587211225071"
2,60A05570770C014AE0530A839427F128,at8,"47.421635044775286,9.373466039585319"
3,E6B834C4D29C023AE0430A839427ECDD,at1,"47.43591444081142,9.403780230280876"
4,E6BE8CDADC3E01D6E0430A839427DF8E,at2,"47.41733179630848,9.367172476537009"
...,...,...,...
1808,165B107D73A30228E0530A8394273D2D,at2,"47.43146199873124,9.381367534662653"
1809,165B90E73BD20072E0530A8394276F3E,at2,"47.421720471300816,9.368895722779616"
1810,14C3BA7D2F55015CE0530A839427E8A8,at2,"47.421393309415706,9.339786621033829"
1811,241F62C9E2E801FAE0530A839427B00B,at2,"47.441698142808185,9.417043095352927"


In [29]:
# Understanding data and correct formatting

accidents.info()
accidents.lat = pd.to_numeric(accidents.lat)
accidents.long = pd.to_numeric(accidents.long)
accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1813 entries, 0 to 1812
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   AccidentUID                  1813 non-null   object
 1   AccidentType                 1813 non-null   object
 2   AccidentType_de              1813 non-null   object
 3   AccidentType_fr              1813 non-null   object
 4   AccidentType_it              1813 non-null   object
 5   AccidentType_en              1813 non-null   object
 6   AccidentSeverityCategory     1813 non-null   object
 7   AccidentSeverityCategory_de  1813 non-null   object
 8   AccidentSeverityCategory_fr  1813 non-null   object
 9   AccidentSeverityCategory_it  1813 non-null   object
 10  AccidentSeverityCategory_en  1813 non-null   object
 11  AccidentInvolvingPedestrian  1813 non-null   bool  
 12  AccidentInvolvingBicycle     1813 non-null   bool  
 13  AccidentInvolvingMotorcycle  1813

In [17]:
# Extracting Location into two separated columns
accidents[['lat','long']] = accidents['Location'].str.split(',',expand=True)
accidents

,AccidentUID,AccidentType,AccidentType_de,AccidentType_fr,AccidentType_it,AccidentType_en,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentSeverityCategory_fr,AccidentSeverityCategory_it,...,AccidentMonth_en,AccidentWeekDay,AccidentWeekDay_de,AccidentWeekDay_fr,AccidentWeekDay_it,AccidentWeekDay_en,AccidentHour,AccidentHour_text,lat,long
0,5CC0CF832F640198E0530A83942715F8,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-10-01,aw402,Dienstag,mardi,Martedì,01-01-02,16,16h-17h,47.438312915107666,9.401879723536387
1,5EA7539566F001CCE0530A839427A876,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as2,Unfall mit Schwerverletzten,accident avec blessés graves,Incidente con feriti gravi,...,01-10-01,aw405,Freitag,vendredi,Venerdì,01-01-05,16,16h-17h,47.43746379938151,9.393587211225071
2,60A05570770C014AE0530A839427F128,at8,Fussgängerunfall,accident impliquant des piétons,Incidente con pedoni,Accident involving pedestrian(s),as2,Unfall mit Schwerverletzten,accident avec blessés graves,Incidente con feriti gravi,...,01-10-01,aw406,Samstag,samedi,Sabato,01-01-06,19,19h-20h,47.421635044775286,9.373466039585319
3,E6B834C4D29C023AE0430A839427ECDD,at1,Überholunfall oder Fahrstreifenwechsel,accident lors d'un dépassement ou lors d'un ch...,Incidente di sorpasso o al cambiamento di corsia,Accident when overtaking or changing lanes,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-08-01,aw403,Mittwoch,mercredi,Mercoledì,01-01-03,17,17h-18h,47.43591444081142,9.403780230280876
4,E6BE8CDADC3E01D6E0430A839427DF8E,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-08-01,aw404,Donnerstag,jeudi,Giovedì,01-01-04,8,08h-09h,47.41733179630848,9.367172476537009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,165B107D73A30228E0530A8394273D2D,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-03-01,aw405,Freitag,vendredi,Venerdì,01-01-05,17,17h-18h,47.43146199873124,9.381367534662653
1809,165B90E73BD20072E0530A8394276F3E,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-04-01,aw401,Montag,lundi,Lunedí,NaN,15,15h-16h,47.421720471300816,9.368895722779616
1810,14C3BA7D2F55015CE0530A839427E8A8,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-04-01,aw403,Mittwoch,mercredi,Mercoledì,01-01-03,11,11h-12h,47.421393309415706,9.339786621033829
1811,241F62C9E2E801FAE0530A839427B00B,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,01-10-01,aw401,Montag,lundi,Lunedí,NaN,7,07h-08h,47.441698142808185,9.417043095352927


In [33]:
# Creating points from coordinates

def create_geometry(row):
  row["geometry"] = Point(row["lat"],row["long"])
  return row

accidents = accidents.apply(create_geometry,axis=1)
accidents_gdf = gpd.GeoDataFrame(accidents,crs=4326)

accidents_gdf

,AccidentUID,AccidentType,AccidentType_de,AccidentType_fr,AccidentType_it,AccidentType_en,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentSeverityCategory_fr,AccidentSeverityCategory_it,...,AccidentWeekDay,AccidentWeekDay_de,AccidentWeekDay_fr,AccidentWeekDay_it,AccidentWeekDay_en,AccidentHour,AccidentHour_text,lat,long,geometry
0,5CC0CF832F640198E0530A83942715F8,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw402,Dienstag,mardi,Martedì,01-01-02,16,16h-17h,47.438313,9.401880,POINT (47.43831 9.40188)
1,5EA7539566F001CCE0530A839427A876,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as2,Unfall mit Schwerverletzten,accident avec blessés graves,Incidente con feriti gravi,...,aw405,Freitag,vendredi,Venerdì,01-01-05,16,16h-17h,47.437464,9.393587,POINT (47.43746 9.39359)
2,60A05570770C014AE0530A839427F128,at8,Fussgängerunfall,accident impliquant des piétons,Incidente con pedoni,Accident involving pedestrian(s),as2,Unfall mit Schwerverletzten,accident avec blessés graves,Incidente con feriti gravi,...,aw406,Samstag,samedi,Sabato,01-01-06,19,19h-20h,47.421635,9.373466,POINT (47.42164 9.37347)
3,E6B834C4D29C023AE0430A839427ECDD,at1,Überholunfall oder Fahrstreifenwechsel,accident lors d'un dépassement ou lors d'un ch...,Incidente di sorpasso o al cambiamento di corsia,Accident when overtaking or changing lanes,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw403,Mittwoch,mercredi,Mercoledì,01-01-03,17,17h-18h,47.435914,9.403780,POINT (47.43591 9.40378)
4,E6BE8CDADC3E01D6E0430A839427DF8E,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw404,Donnerstag,jeudi,Giovedì,01-01-04,8,08h-09h,47.417332,9.367172,POINT (47.41733 9.36717)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1808,165B107D73A30228E0530A8394273D2D,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw405,Freitag,vendredi,Venerdì,01-01-05,17,17h-18h,47.431462,9.381368,POINT (47.43146 9.38137)
1809,165B90E73BD20072E0530A8394276F3E,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw401,Montag,lundi,Lunedí,NaN,15,15h-16h,47.421720,9.368896,POINT (47.42172 9.36890)
1810,14C3BA7D2F55015CE0530A839427E8A8,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw403,Mittwoch,mercredi,Mercoledì,01-01-03,11,11h-12h,47.421393,9.339787,POINT (47.42139 9.33979)
1811,241F62C9E2E801FAE0530A839427B00B,at2,Auffahrunfall,accident par tamponnement,Incidente di tamponamento,Accident with rear-end collision,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,...,aw401,Montag,lundi,Lunedí,NaN,7,07h-08h,47.441698,9.417043,POINT (47.44170 9.41704)


In [37]:
bus_stops = pd.read_csv("haltestelle-didok.csv", sep=';')
bus_stops

,nummer,BPUIC,Name Haltestelle,Abkuerzung Bahnhof,ORTSCHAFTSNAME,GEMEINDENAME,BFS Nummer Gemeinde,Bezirksname,BFS Nummer Bezirk,Name des Kanton,...,GO_IDENTIFIKATION,tu-nummer,GO_ABKUERZUNG_DE,GO_BEZEICHNUNG_DE,lod,Geoposition,Ist Haltestelle Flag,IS_BEDIENPUNKT,IS_VERKEHRSPUNKT,plz
0,74284,8574284,"Rorschacherberg, Zelgstrasse",NaN,Rorschacherberg,Rorschacherberg,3216,Rorschach,1722,St. Gallen,...,100602,801,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8574284,"47.47066921504456,9.48726485400962",1,0,1,9404
1,73859,8573859,"Eschenbach SG, Mettlen",NaN,Eschenbach SG,Eschenbach (SG),3342,See-Gaster,1726,St. Gallen,...,100602,801,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573859,"47.237581294155454,8.923876933092943",1,0,1,8733
2,73848,8573848,"Ernetschwil, Häblingen",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,100602,801,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573848,"47.24199136341773,8.9966026079072",1,0,1,8725
3,73844,8573844,"Ernetschwil, Dorf",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,100602,801,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573844,"47.23737047712995,9.00101180431625",1,0,1,8725
4,74263,8574263,"Untereggen, Vogelherd",NaN,Untereggen,Untereggen,3219,Rorschach,1722,St. Gallen,...,100602,801,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8574263,"47.459324519147614,9.457045509530223",1,0,1,9033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,16197,8516197,Nesslau-Neu St. Joh. Gleisende,NLGE,Nesslau,Nesslau,3360,Toggenburg,1727,St. Gallen,...,100018,36,SOB-bt,Schweizerische Südostbahn (bt),http://lod.opentransportdata.swiss/didok/8516197,"47.22562011815448,9.198896755703965",0,0,0,9650
1710,6294,8506294,Brunnadern-Neckertal,BRN,Brunnadern,Neckertal,3378,Toggenburg,1727,St. Gallen,...,100018,36,SOB-bt,Schweizerische Südostbahn (bt),http://lod.opentransportdata.swiss/didok/8506294,"47.33782748620906,9.130595381956109",1,1,1,9125
1711,89624,8589624,"St. Gallen, Seeblick",NaN,St. Gallen,St. Gallen,3203,St. Gallen,1721,St. Gallen,...,100684,885,VBSG,Verkehrsbetriebe der Stadt St.Gallen,http://lod.opentransportdata.swiss/didok/8589624,"47.43994306691554,9.377791746273482",1,0,1,9010
1712,9405,8509405,Räfis-Burgerau,RAEF,Buchs SG,Buchs (SG),3271,Werdenberg,1724,St. Gallen,...,100001,11,SBB,Schweizerische Bundesbahnen SBB,http://lod.opentransportdata.swiss/didok/8509405,"47.14994819461831,9.485185119142594",0,0,0,9470


In [39]:
# Extracting Location into two separated columns and creating two separated columns

bus_stops[['lat','long']] = bus_stops['Geoposition'].str.split(',',expand=True)
bus_stops

,nummer,BPUIC,Name Haltestelle,Abkuerzung Bahnhof,ORTSCHAFTSNAME,GEMEINDENAME,BFS Nummer Gemeinde,Bezirksname,BFS Nummer Bezirk,Name des Kanton,...,GO_ABKUERZUNG_DE,GO_BEZEICHNUNG_DE,lod,Geoposition,Ist Haltestelle Flag,IS_BEDIENPUNKT,IS_VERKEHRSPUNKT,plz,lat,long
0,74284,8574284,"Rorschacherberg, Zelgstrasse",NaN,Rorschacherberg,Rorschacherberg,3216,Rorschach,1722,St. Gallen,...,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8574284,"47.47066921504456,9.48726485400962",1,0,1,9404,47.47066921504456,9.48726485400962
1,73859,8573859,"Eschenbach SG, Mettlen",NaN,Eschenbach SG,Eschenbach (SG),3342,See-Gaster,1726,St. Gallen,...,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573859,"47.237581294155454,8.923876933092943",1,0,1,8733,47.237581294155454,8.923876933092943
2,73848,8573848,"Ernetschwil, Häblingen",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573848,"47.24199136341773,8.9966026079072",1,0,1,8725,47.24199136341773,8.9966026079072
3,73844,8573844,"Ernetschwil, Dorf",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573844,"47.23737047712995,9.00101180431625",1,0,1,8725,47.23737047712995,9.00101180431625
4,74263,8574263,"Untereggen, Vogelherd",NaN,Untereggen,Untereggen,3219,Rorschach,1722,St. Gallen,...,PAG,PostAuto AG,http://lod.opentransportdata.swiss/didok/8574263,"47.459324519147614,9.457045509530223",1,0,1,9033,47.459324519147614,9.457045509530223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,16197,8516197,Nesslau-Neu St. Joh. Gleisende,NLGE,Nesslau,Nesslau,3360,Toggenburg,1727,St. Gallen,...,SOB-bt,Schweizerische Südostbahn (bt),http://lod.opentransportdata.swiss/didok/8516197,"47.22562011815448,9.198896755703965",0,0,0,9650,47.22562011815448,9.198896755703965
1710,6294,8506294,Brunnadern-Neckertal,BRN,Brunnadern,Neckertal,3378,Toggenburg,1727,St. Gallen,...,SOB-bt,Schweizerische Südostbahn (bt),http://lod.opentransportdata.swiss/didok/8506294,"47.33782748620906,9.130595381956109",1,1,1,9125,47.33782748620906,9.130595381956109
1711,89624,8589624,"St. Gallen, Seeblick",NaN,St. Gallen,St. Gallen,3203,St. Gallen,1721,St. Gallen,...,VBSG,Verkehrsbetriebe der Stadt St.Gallen,http://lod.opentransportdata.swiss/didok/8589624,"47.43994306691554,9.377791746273482",1,0,1,9010,47.43994306691554,9.377791746273482
1712,9405,8509405,Räfis-Burgerau,RAEF,Buchs SG,Buchs (SG),3271,Werdenberg,1724,St. Gallen,...,SBB,Schweizerische Bundesbahnen SBB,http://lod.opentransportdata.swiss/didok/8509405,"47.14994819461831,9.485185119142594",0,0,0,9470,47.14994819461831,9.485185119142594


In [41]:
bus_stops.info()

bus_stops.lat = pd.to_numeric(bus_stops.lat)
bus_stops.long = pd.to_numeric(bus_stops.long)

bus_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   nummer                     1714 non-null   int64  
 1   BPUIC                      1714 non-null   int64  
 2   Name Haltestelle           1714 non-null   object 
 3   Abkuerzung Bahnhof         381 non-null    object 
 4   ORTSCHAFTSNAME             1714 non-null   object 
 5   GEMEINDENAME               1714 non-null   object 
 6   BFS Nummer Gemeinde        1714 non-null   int64  
 7   Bezirksname                1714 non-null   object 
 8   BFS Nummer Bezirk          1714 non-null   int64  
 9   Name des Kanton            1714 non-null   object 
 10  Kantons Abkürzung          1714 non-null   object 
 11  Kanton (BFS)               1714 non-null   int64  
 12  LAND_ISO2_GEO              1714 non-null   object 
 13  BP_ART_BEZEICHNUNG_DE      36 non-null     objec

In [43]:
# Creating points from coordinates
bus_stops = bus_stops.apply(create_geometry,axis=1)
bus_stops_gdf = gpd.GeoDataFrame(bus_stops,crs=4326)

bus_stops_gdf

,nummer,BPUIC,Name Haltestelle,Abkuerzung Bahnhof,ORTSCHAFTSNAME,GEMEINDENAME,BFS Nummer Gemeinde,Bezirksname,BFS Nummer Bezirk,Name des Kanton,...,GO_BEZEICHNUNG_DE,lod,Geoposition,Ist Haltestelle Flag,IS_BEDIENPUNKT,IS_VERKEHRSPUNKT,plz,lat,long,geometry
0,74284,8574284,"Rorschacherberg, Zelgstrasse",NaN,Rorschacherberg,Rorschacherberg,3216,Rorschach,1722,St. Gallen,...,PostAuto AG,http://lod.opentransportdata.swiss/didok/8574284,"47.47066921504456,9.48726485400962",1,0,1,9404,47.470669,9.487265,POINT (47.47067 9.48726)
1,73859,8573859,"Eschenbach SG, Mettlen",NaN,Eschenbach SG,Eschenbach (SG),3342,See-Gaster,1726,St. Gallen,...,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573859,"47.237581294155454,8.923876933092943",1,0,1,8733,47.237581,8.923877,POINT (47.23758 8.92388)
2,73848,8573848,"Ernetschwil, Häblingen",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573848,"47.24199136341773,8.9966026079072",1,0,1,8725,47.241991,8.996603,POINT (47.24199 8.99660)
3,73844,8573844,"Ernetschwil, Dorf",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,PostAuto AG,http://lod.opentransportdata.swiss/didok/8573844,"47.23737047712995,9.00101180431625",1,0,1,8725,47.237370,9.001012,POINT (47.23737 9.00101)
4,74263,8574263,"Untereggen, Vogelherd",NaN,Untereggen,Untereggen,3219,Rorschach,1722,St. Gallen,...,PostAuto AG,http://lod.opentransportdata.swiss/didok/8574263,"47.459324519147614,9.457045509530223",1,0,1,9033,47.459325,9.457046,POINT (47.45932 9.45705)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,16197,8516197,Nesslau-Neu St. Joh. Gleisende,NLGE,Nesslau,Nesslau,3360,Toggenburg,1727,St. Gallen,...,Schweizerische Südostbahn (bt),http://lod.opentransportdata.swiss/didok/8516197,"47.22562011815448,9.198896755703965",0,0,0,9650,47.225620,9.198897,POINT (47.22562 9.19890)
1710,6294,8506294,Brunnadern-Neckertal,BRN,Brunnadern,Neckertal,3378,Toggenburg,1727,St. Gallen,...,Schweizerische Südostbahn (bt),http://lod.opentransportdata.swiss/didok/8506294,"47.33782748620906,9.130595381956109",1,1,1,9125,47.337827,9.130595,POINT (47.33783 9.13060)
1711,89624,8589624,"St. Gallen, Seeblick",NaN,St. Gallen,St. Gallen,3203,St. Gallen,1721,St. Gallen,...,Verkehrsbetriebe der Stadt St.Gallen,http://lod.opentransportdata.swiss/didok/8589624,"47.43994306691554,9.377791746273482",1,0,1,9010,47.439943,9.377792,POINT (47.43994 9.37779)
1712,9405,8509405,Räfis-Burgerau,RAEF,Buchs SG,Buchs (SG),3271,Werdenberg,1724,St. Gallen,...,Schweizerische Bundesbahnen SBB,http://lod.opentransportdata.swiss/didok/8509405,"47.14994819461831,9.485185119142594",0,0,0,9470,47.149948,9.485185,POINT (47.14995 9.48519)


In [49]:
def point2circle(row):
    radius = 200
    
    local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(
        row.lat, row.long
    )
    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
        pyproj.Proj(local_azimuthal_projection),
    )
    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
    )

    center = Point(float(row.lat), float(row.long))
    point_transformed = transform(wgs84_to_aeqd, center)
    buffer = point_transformed.buffer(radius)
    # Get the polygon with lat lon coordinates
    circle_poly = transform(aeqd_to_wgs84, buffer)
    row['Area_circle'] = circle_poly
    return row

bus_stops = bus_stops.apply(point2circle,axis=1)

In [50]:
bus_stops.head()

,nummer,BPUIC,Name Haltestelle,Abkuerzung Bahnhof,ORTSCHAFTSNAME,GEMEINDENAME,BFS Nummer Gemeinde,Bezirksname,BFS Nummer Bezirk,Name des Kanton,...,lod,Geoposition,Ist Haltestelle Flag,IS_BEDIENPUNKT,IS_VERKEHRSPUNKT,plz,lat,long,geometry,Area_circle
0,74284,8574284,"Rorschacherberg, Zelgstrasse",NaN,Rorschacherberg,Rorschacherberg,3216,Rorschach,1722,St. Gallen,...,http://lod.opentransportdata.swiss/didok/8574284,"47.47066921504456,9.48726485400962",1,0,1,9404,47.470669,9.487265,POINT (47.47066921504456 9.48726485400962),"POLYGON ((47.47227450040047 9.486587073411409,..."
1,73859,8573859,"Eschenbach SG, Mettlen",NaN,Eschenbach SG,Eschenbach (SG),3342,See-Gaster,1726,St. Gallen,...,http://lod.opentransportdata.swiss/didok/8573859,"47.237581294155454,8.923876933092943",1,0,1,8733,47.237581,8.923877,POINT (47.237581294155454 8.923876933092943),"POLYGON ((47.23918284524313 8.923199081620544,..."
2,73848,8573848,"Ernetschwil, Häblingen",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,http://lod.opentransportdata.swiss/didok/8573848,"47.24199136341773,8.9966026079072",1,0,1,8725,47.241991,8.996603,POINT (47.24199136341773 8.9966026079072),"POLYGON ((47.24359375250619 8.995925492763917,..."
3,73844,8573844,"Ernetschwil, Dorf",NaN,Ernetschwil,Gommiswald,3341,See-Gaster,1726,St. Gallen,...,http://lod.opentransportdata.swiss/didok/8573844,"47.23737047712995,9.00101180431625",1,0,1,8725,47.237370,9.001012,POINT (47.23737047712995 9.00101180431625),"POLYGON ((47.23897298708682 9.000334873643085,..."
4,74263,8574263,"Untereggen, Vogelherd",NaN,Untereggen,Untereggen,3219,Rorschach,1722,St. Gallen,...,http://lod.opentransportdata.swiss/didok/8574263,"47.459324519147614,9.457045509530223",1,0,1,9033,47.459325,9.457046,POINT (47.459324519147614 9.457045509530223),POLYGON ((47.460929582232396 9.456367688765726...
